In [7]:
import pandas as pd
import numpy as np

# 0) 데이터 준비

hackle = pd.read_csv("clean_vote_ver2/processed_hackle_merge.csv")
pay = pd.read_csv("clean_vote_ver2/processed_accounts_paymenthistory.csv")

/var/folders/v1/k2x8lm750mbb_z0sj197vzrh0000gn/T/ipykernel_16147/3247948985.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  hackle = pd.read_csv("clean_vote_ver2/processed_hackle_merge.csv")


In [3]:
hackle["event_datetime"] = pd.to_datetime(hackle["event_datetime"]).dt.floor("s")
hackle = hackle.sort_values(["user_id", "event_datetime"])

In [4]:
eng = (hackle[hackle["event_key"] == "complete_question"].groupby("user_id")["event_datetime"].min().reset_index(name="eng_first"))

In [5]:
last_log = (hackle.groupby("user_id")["event_datetime"].max().reset_index(name="last_log"))

In [8]:
pay["created_at"] = pd.to_datetime(pay["created_at"]).dt.floor("s")

pay_user = (pay.groupby("user_id")["created_at"].min().reset_index(name="pay_first"))

In [9]:
intent_keys = {"view_shop", "click_purchase"}

intent_user = (hackle[hackle["event_key"].isin(intent_keys)].groupby("user_id")["event_datetime"].min().reset_index(name="intent_first"))

In [10]:
tmp = eng.merge(last_log, on="user_id", how="left")
tmp = tmp.merge(intent_user, on="user_id", how="left")
tmp = tmp.merge(pay_user, on="user_id", how="left")

In [11]:
# 참여 이후 추가 로그 존재
tmp["is_retained"] = tmp["last_log"] > tmp["eng_first"]

# 참여 이후 상점/구매 클릭
tmp["is_intent"] = tmp["intent_first"] > tmp["eng_first"]

# 진짜 매출: 참여 이후 결제기록 존재
tmp["is_revenue"] = tmp["pay_first"] > tmp["eng_first"]

In [12]:
eng_cnt = len(tmp)
ret_cnt = tmp["is_retained"].sum()
intent_cnt = tmp["is_intent"].sum()
rev_cnt = tmp["is_revenue"].sum()

print("---Funnel---")
print(f"참여 유저: {eng_cnt:,}")
print(f"리텐션 유저: {ret_cnt:,} | 전환율: {ret_cnt/eng_cnt:.2%}")
print(f"구매의도 유저: {intent_cnt:,} | 전환율(참여 기준): {intent_cnt/eng_cnt:.2%}")
print(f"구매완료 유저(실매출): {rev_cnt:,} | 전환율(참여 기준): {rev_cnt/eng_cnt:.2%}")

---Funnel---
참여 유저: 48,981
리텐션 유저: 46,030 | 전환율: 93.98%
구매의도 유저: 5,766 | 전환율(참여 기준): 11.77%
구매완료 유저(실매출): 676 | 전환율(참여 기준): 1.38%


In [ ]:
# 구매 기록 내부데이터로 확인
# 구매 의도 단계 추가